In [50]:
import cv2
import os
print(os.getcwd())
import sys
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from numpy import genfromtxt
from fr_utils import *
from inception_blocks_v2 import *
from tensorflow.compat.v1.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras.layers import BatchNormalization
from tensorflow.compat.v1.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.compat.v1.keras.layers import Concatenate
from tensorflow.compat.v1.keras.layers import Lambda, Flatten, Dense
from tensorflow.compat.v1.keras.initializers import glorot_uniform
from tensorflow.compat.v1.keras.layers import Layer
from tensorflow.compat.v1.keras import backend as K
K.set_image_data_format('channels_first')

/content


In [51]:
print(tf.__version__)
%matplotlib inline
%load_ext autoreload
%autoreload 2

2.5.0
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [53]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


$Triplet Loss$


In [54]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis = -1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis = -1)
    basic_loss = pos_dist- neg_dist + alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    return loss

In [55]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    print("loss = " + str(loss.eval()))

loss = 528.5425


In [56]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [57]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

In [58]:
def verify(image_path, identity, database, model):
    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm(encoding - database[identity])
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False        
    return dist, door_open

In [59]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

It's younes, welcome in!


(0.67100644, True)

In [60]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

It's not kian, please go away


(0.8580015, False)

In [61]:
def who_is_it(image_path, database, model):
    encoding = img_to_encoding(image_path, model)
    min_dist = 100
    for (name, db_enc) in database.items():
      dist = np.linalg.norm(encoding-db_enc)
    if dist < min_dist:
            min_dist = dist
            identity = name
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
    return min_dist, identity

In [62]:
who_is_it("images/camera_0.jpg", database, FRmodel)

Not in the database.


(0.83424747, 'arnaud')

In [63]:
who_is_it("images/camera_1.jpg", database, FRmodel)
who_is_it("images/camera_2.jpg", database, FRmodel)
who_is_it("images/camera_3.jpg", database, FRmodel)
who_is_it("images/camera_4.jpg", database, FRmodel)
who_is_it("images/camera_5.jpg", database, FRmodel)

it's arnaud, the distance is 0.68323416
Not in the database.
it's arnaud, the distance is 0.57619536
it's arnaud, the distance is 0.5293446
it's arnaud, the distance is 0.5988951


(0.5988951, 'arnaud')